In [ ]:
# useful hack: do this to restart the kernel (and clear cuda memory) from within the notebook
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

**PRELIMINARIES Step 0: Needed for all additional steps, define functions used:**

In [ ]:
from numpy import *
import pickle
import torch
import re

alphabet = "acgt"
def decode(x): # convert list of ints to string
    s = "".join([alphabet[xx] for xx in x])
    return s
def encode(st): # convert a string into a list of ints
    x = [alphabet.index(ss) for ss in st]
    return x
def seqtomer(seq) : # return list of trimers in a seq
    ans = [int(16*seq[k] + 4*seq[k+1] + seq[k+2]) for k in range(len(seq)-2)]
    return ans
def mertobin(mer) : # trimer list to occupancy uint64 bitmap
    ibin = 0
    for m in mer :
        ibin |= (1 << m)
    return ibin

def makeerrors(seq,srate,irate,drate) : #error mode applied to a sequence seq
    # note: modifies (also returns) seq
    n = len(seq)
    # substitutions
    ns = random.binomial(n,srate*1.3333) # 3/4 of substitutions are "effective"
    ndx = random.randint(low=0,high=n,size=ns)
    vals = random.randint(low=0,high=4,size=ns)
    seq[ndx] = vals
    # deletions
    nd = random.binomial(n,drate)
    ndx = random.randint(low=0,high=n,size=nd)
    seq = delete(seq,ndx)    
    # insertions (at specified rate into smaller seq)
    ni = random.binomial(len(seq),irate)
    ndx = random.randint(low=0,high=len(seq)+1,size=ni)
    vals = random.randint(low=0,high=4,size=ni)
    seq = insert(seq,ndx,vals)
    # pad or truncate to original length
    nn = len(seq)
    if nn > n :
        seq = seq[:n]
    elif nn < n :
        seq = concatenate((seq,random.randint(low=0,high=4,size=n-nn)))
    return seq

m0 = uint64(0x5555555555555555)  # binary: 0101...
m1 = uint64(0x3333333333333333)  # binary: 00110011..
m2 = uint64(0x0f0f0f0f0f0f0f0f)  # binary:  4 zeros,  4 ones ...
m3 = uint64(0x00ff00ff00ff00ff)  # binary:  8 zeros,  8 ones ...
m4 = uint64(0x0000ffff0000ffff)  # binary: 16 zeros, 16 ones ...
m5 = uint64(0x00000000ffffffff)  # binary: 32 zeros, 32 ones
def popcount(x):
# https://github.com/google/jax/blob/6c8fc1b031275c85b02cb819c6caa5afa002fa1d/jax/lax_reference.py#L121-L150
    x = (x & m0) + ((x >>  1) & m0)  # put count of each  2 bits into those  2 bits
    x = (x & m1) + ((x >>  2) & m1)  # put count of each  4 bits into those  4 bits
    x = (x & m2) + ((x >>  4) & m2)  # put count of each  8 bits into those  8 bits
    x = (x & m3) + ((x >>  8) & m3)  # put count of each 16 bits into those 16 bits
    x = (x & m4) + ((x >> 16) & m4)  # put count of each 32 bits into those 32 bits
    x = (x & m5) + ((x >> 32) & m5)  # put count of each 64 bits into those 64 bits
    return x

try :
    from popcll_torch import popcll
    #https://github.com/iamgroot42/popcll_torch , thanks to Anshuman Suri (as9rw@virginia.edu)!
    mypopcount = popcll
except :
    mypopcount = popcount
    print("popcll_torch not found, so using slower popcount")

def find_runs(x):
    #Find runs of consecutive items in an array.
    # credit: https://gist.github.com/alimanfoo/c5977e87111abe8127453b21204c1065
    n = x.shape[0]
    # find run starts
    loc_run_start = empty(n, dtype=bool)
    loc_run_start[0] = True
    not_equal(x[:-1], x[1:], out=loc_run_start[1:])
    run_starts = nonzero(loc_run_start)[0]
    # find run values
    run_values = x[loc_run_start]
    # find run lengths
    run_lengths = diff(append(run_starts, n))
    return run_values, run_starts, run_lengths

def chemfilter(seq, homomax=3, atmax=22, cgmax=22) :
    # returns whether seq satisfies chemistry constraints
    bc = bincount(seq,minlength=4)
    if (bc[0]+bc[3] > atmax) or (bc[1]+bc[2] > cgmax) : return False
    _,_,run_lengths = find_runs(seq)
    if max(run_lengths) > homomax : return False
    return True

def allcoses(mer, tcosvecs) : # correlate a mer against all the cosine templates
    mmer = torch.LongTensor(mer).to(device)
    ncos = tcosvecs.shape[0]
    cosvec = torch.zeros(ncos, 64, dtype=torch.float, device=device)
    for k in range(ncos) :
        source = tcoses[k, torch.arange(len(mmer), dtype=torch.long, device=device)]
        cosvec[k,:].index_add_(0,mmer,source)
    return torch.sum(torch.unsqueeze(cosvec,dim=1)*tcosvecs,dim=2)

def prank(arr, descending=False) : # returns rank of each element in torch array
    argsrt = torch.argsort(arr, descending=descending)
    rank = torch.zeros(arr.shape, dtype=torch.float, device=device)
    rank[argsrt] = torch.arange(len(argsrt),dtype=torch.float,device=device)
    return rank    

class ApproximateLevenshtein :
    def __init__(s, M, N, Q, zsub, zins, zdel, zskew):
        torch.set_grad_enabled(False) # just in case not done elsewhere!
        s.M = M # length of seq1
        s.N = N # length of each seq2
        s.Q = Q # number of seq2s
        (s.zsub, s.zins, s.zdel, s.zskew) = (zsub, zins, zdel, zskew)
        s.tab = torch.zeros(N+1,Q, device=device)
        
    def __call__(s,seq1,seq2) :
        assert (len(seq1) == s.M) and (seq2.shape[1] == s.N) and (seq2.shape[0] == s.Q)
        s.tab[:,:] = (s.zskew * torch.arange(s.N+1., device=device)).unsqueeze(1) # force broadcast
        for i in range(1,s.M+1) :
            diag = s.tab[:-1,:] + torch.where(seq1[i-1] == seq2.t(), 0., s.zsub) # diagonal move
            s.tab[0,:] += s.zskew
            s.tab[1:,:] += s.zdel # down move
            s.tab[1:,:] = torch.minimum(s.tab[1:,:], diag) # or diag if better
            s.tab[1:,:] = torch.minimum(s.tab[1:,:], s.tab[:-1,:] + s.zins) # right move
            s.tab[1:,:] = torch.minimum(s.tab[1:,:], s.tab[:-1,:] + s.zins) # repeat (>= 0 times) as you can afford
           # N.B.: M >= N gives better approx than N > M, so change arg order accordingly
        return s.tab[s.N,:]

class ParallelLevenshtein :
    def __init__(s, M, N, Q, zsub, zins, zdel, zskew):
        torch.set_grad_enabled(False) # just in case not done elsewhere!
        s.M = M # length of seq1
        s.N = N # length of each seq2
        s.Q = Q # number of seq2s
        (s.zsub, s.zins, s.zdel, s.zskew) = (zsub, zins, zdel, zskew)
        MN1 = M + N + 1
        s.blue = torch.zeros(Q, MN1, MN1, device=device)
        s.bluef = s.blue.view(Q, MN1 * MN1)
        s.ndxr = torch.zeros(M*N, dtype=int, device=device) # index of mer matches array into flat blue
        for m in torch.arange(M,device=device) :
            for n in torch.arange(N,device=device) :
                s.ndxr[n + N*m] = (3*M+2*N+2) + (M+N)*m + (M+N+2)*n
        s.lls = torch.zeros(MN1+1,dtype=torch.int,device=device)       
        s.rrs = torch.zeros(MN1+1,dtype=torch.int,device=device)       
        for i in range(2,MN1+1) :
            s.lls[i] = abs(M - i + 1) + 1
            s.rrs[i] = (M+N-1) - abs(- i + 1 + N )

    def __call__(s, seq1, sseq2): # single seq1, tensor of sseq2s
        assert (len(seq1) == s.M) and (sseq2.shape[1] == s.N) and (sseq2.shape[0] == s.Q)    
        (M1,N1,MN,MN1,MN2) = (s.M + 1, s.N + 1, s.M + s.N, s.M + s.N + 1, s.M + s.N + 2)
        abmatch = (seq1.view(1,s.M,1) != sseq2.view(s.Q,1,s.N)).type(torch.float) * s.zsub
        s.bluef[:,s.ndxr] = abmatch.view(s.Q,s.M*s.N)
        s.bluef[:,torch.arange(s.M,MN2*N1,MN2)] = (s.zskew*torch.arange(N1,device=device)).unsqueeze(0)
        s.bluef[:,torch.arange(s.M,MN*M1,MN)] = (s.zskew*torch.arange(M1,device=device)).unsqueeze(0)
        for k in torch.arange(2,MN1,device=device) :
            ll = s.lls[k]
            rr = s.rrs[k]
            slis = torch.arange(ll,rr+1,2,device=device)
            s.blue[:,k,slis] = torch.minimum(
                s.blue[:,k,slis] + s.blue[:,k-2,slis],
                torch.minimum(
                    s.blue[:,k-1,slis-1] + s.zdel,
                    s.blue[:,k-1,slis+1] + s.zins
                )
            )
        return s.blue[:,-1,s.N]

print("all functions now defined.")

Expected output above is
`all functions now defined.`\
If you see `popcll_torch not found, so using slower popcount`
then you may want to install popcll_torch from
[here](https://github.com/iamgroot42/popcll_torch)
for a modest speed improvement and smaller GPU memory use

Next: **PRELIMINARIES: Do either Step 1a *or* Step 1b** to create a barcode library or use an existing (saved) one 

In [ ]:
%%time
# Step 1a: create to a set of random codewords (do just once)
# writes to 2 files: ascii acgt strings, and (with auxilliary tables) Python pickle  

N = 100000 # user to set to number of codewords (this is a small value for demo purposes, 1000000 is feasible)
M = 34 # user to set to length of a codeword (nt)
filename = "/tmp/randomcode_wchem_1e5_34" # user set to output file path (produces file length ~ M*N)
picklefilename = "/tmp/randomcode_wchem_1e5_34.pkl" # user to set (produces file length ~64*M*N)
Ncos = 4 # user set to number of cosine templates (usually 4)
DoChemFilter = True # filter for homopolymer runs, AT and CG content?
(homomax, atmax, cgmax) = (3, int(0.66*M), int(0.66*M)) # user to set if DoChemFilter is True

# generate and write the codes
torch.set_grad_enabled(False)
if DoChemFilter :
    fac = 2. # should work in all expected cases
    Ntrial = int(fac*N)
    allcands = random.randint(low=0,high=4,size=(Ntrial,M))
    goodcands = array([chemfilter(x,homomax,atmax,cgmax) for x in allcands], dtype=bool)
    allcodes = allcands[goodcands][:N] # if this or next throws an error, increase fac, but shouldn't happen
    if allcodes.shape != (N,M) : raise RuntimeError("see code, increase fac (this shouldn\'t happen')")
else :
    allcodes = random.randint(low=0,high=4,size=(N,M))
with open(filename, 'w') as OUT:
    for code in allcodes :        
        OUT.write(decode(code) + '\n')
print(f"wrote {N} codewords of length {M} in ascii to {filename}, now computing auxilliary tables")

# re-read codes to be sure that the pickle file matches the ascii file!
with open(filename) as IN: 
    codes=IN.readlines() # have \n EOL
assert N == len(codes)
assert M == len(codes[0][:-1])
allseqs = []
alltrimers = []
allbitmaps = zeros(N, dtype=uint64)
cosvecs = torch.zeros((Ncos,N,64),dtype=torch.float)
coses = zeros((Ncos,M)) 
for k in range(Ncos) :
    coses[k,:] = cos(pi*arange(M)*(k+1.)/(M-1.))
tcoses = torch.tensor(coses, dtype=torch.float)
for i,code in enumerate(codes) :
    seq = encode(code[:-1])
    allseqs.append(seq)
    mer = seqtomer(seq)
    mmer = torch.LongTensor(mer)
    alltrimers.append(mer)
    allbitmaps[i] = mertobin(mer)
    for k in range(Ncos):
        source = tcoses[k,arange(M-2)]
        cosvecs[k,i,:].index_add_(0,mmer,source)
print("finished making code auxilliary tables, now pickling")        
pickledict = {"N" : N, "M" : M, "allseqs" : allseqs,
    "alltrimers" : alltrimers, "allbitmaps" : allbitmaps, "coses" : coses, "cosvecs" : cosvecs}
with open(picklefilename,'wb') as OUT :
    pickle.dump(pickledict,OUT)
print(f"finished pickling code with {N} codewords of length {M} to {picklefilename}")        


Typical output above is\
`wrote 100000 codewords of length 34 in ascii to /tmp/randomcode_wchem_1e5_34, now computing auxilliary tables
finished making code auxilliary tables, now pickling
finished pickling code with 100000 codewords of length 34 to /tmp/randomcode_wchem_1e5_34.pkl
CPU times: user 5.43 s, sys: 326 ms, total: 5.76 s
Wall time: 5.77 s`

In [ ]:
%%time
# *or* Step 1b: load code from its pickle file

picklefilename = "/tmp/randomcode_wchem_1e5_34.pkl" # user to set
with open(picklefilename,'rb') as IN :
    pickledict = pickle.load(IN)
(N, M, allseqs, alltrimers, allbitmaps, coses, cosvecs) = \
    [pickledict[x] for x in ('N', 'M', 'allseqs', 'alltrimers', 'allbitmaps', 'coses', 'cosvecs')]
print(f"loaded code with {N} codewords of length {M} from {picklefilename}.")

Typical output above is\
`loaded code with 100000 codewords of length 34 from /tmp/randomcode_wchem_1e5_34.pkl.
CPU times: user 372 ms, sys: 95.9 ms, total: 468 ms
Wall time: 464 ms`

**Next: PRELIMINARIES Step 2, make simulated reads**

In [ ]:
%%time
# PRELIMINARIES Step 2: make simulated data, write "reads" to ascii file,
# "answers (indices of true codeword)" to another file. Assumes code is loaded by Step 1a or 1b.

Q = 10000 # user to set to desired number of simulated reads (10000 is a small value for demo purposes)
nave = 4. # user to set (average Poisson number of reads from each randomly selected codeword)
(srate,irate,drate) = (0.05, 0.05, 0.10) # user to set (these are *very* large error rates)
#(srate,irate,drate) = (0.0333, 0.0333, 0.0333) # user to set (these are large error rates)
readsfilename = "/tmp/randomcode_1e6_34_sim_reads" # user set to output file path
answersfilename = "/tmp/randomcode_1e6_34_sim_answers" # user set to output file path

q = 0
reads = []
answers = []
while q < Q :
    ansindex = random.randint(low=0,high=N)
    ans = allseqs[ansindex]
    n = min(Q-q, random.poisson(lam=nave))
    for i in range(n) :
        reads.append(makeerrors(copy(ans),srate,irate,drate))
        answers.append(ansindex)
    q += n
with open(readsfilename, 'w') as OUT:
    for seq in reads :        
        OUT.write(decode(seq) + '\n')
with open(answersfilename, 'w') as OUT:
    for ans in answers :        
        OUT.write(str(ans) + '\n')
print(f"done creating {Q} simulated reads and writing to {readsfilename} \n(answers to {answersfilename})")


Typical output above is\
`done creating 10000 simulated reads and writing to /tmp/randomcode_1e6_34_sim_reads
(answers to /tmp/randomcode_1e6_34_sim_answers)
CPU times: user 766 ms, sys: 19.2 ms, total: 785 ms
Wall time: 774 ms`

**Code creation and simulated data steps complete. Now demonstrate decoding (as if from data):**

In [ ]:
%%time
# DECODING Step 1. Move the code (assumed loaded above) and reads (from file) to the GPU.

readsfilename = "/tmp/randomcode_1e6_34_sim_reads" # user to set

if torch.cuda.is_available() :
    device = torch.device("cuda")
    cudaname = torch.cuda.get_device_name()
else :
    raise RuntimeError("Required GPU not found! Exiting.")
print(f"Using {device} device {cudaname}.")

with open(readsfilename, 'r') as IN:
    stringreads=IN.readlines()
reads = []
for read in stringreads :
    reads.append(encode(read[:-1])) # lose the \n
reads = array(reads)

torch.set_grad_enabled(False)
tallseqs = torch.tensor(array(allseqs), device=device)
talltrimers = torch.tensor(array(alltrimers), device=device)
tallbitmaps = torch.tensor(allbitmaps.astype(int64), dtype=torch.int64, device=device) # 
tcoses = torch.tensor(coses, dtype=torch.float, device=device)
tcosvecs = cosvecs.to(device)
print(f"found {reads.shape[0]} reads of length {reads.shape[1]}")

Typical output above is\
`Using cuda device NVIDIA GeForce RTX 3090.
found 10000 reads of length 34
CPU times: user 1.18 s, sys: 429 ms, total: 1.61 s
Wall time: 1.46 s`

**Next: the main decoding step, with almost all the running time**

In [ ]:
%%time
# DECODING Step 2 (main step). Primary and secondary triage, followed by Levenshtein

Qq = 10000 # reads.shape[0] or smaller number, how many reads to do
Ntriage = 10000 # user set to number passed from triage to Levenshtein
Nthresh = 8 # user set to Levenshtein score greater than which is called an erasure

torch.set_grad_enabled(False)

mydist = ApproximateLevenshtein(M,M,Ntriage, 1.,1.,1.,1.)
#mydist = ParallelLevenshtein(M,M,Ntriage, 1.,1.,1.,1.)

Ncos = tcosvecs.shape[0]
dists = torch.zeros(Ncos+1, N, dtype=torch.float, device=device) # will hold distances for each read
allrank = torch.zeros(Ncos+1 ,N, dtype=torch.float, device=device)
best = torch.zeros(Qq, dtype=torch.long, device=device)

for j,seq in enumerate(reads[:Qq]) :
    # primary and secondary triage
    mer = seqtomer(seq)
    foo = int64(uint64(mertobin(mer))) # need to cast 64 bits to a type known to torch
    seqbin = torch.tensor(foo,dtype=torch.int64,device=device)
    xored = torch.bitwise_xor(seqbin,tallbitmaps)
    dists[0,:] = 64. - mypopcount(xored) # all Hamming distances
    cosvec = torch.zeros(Ncos, 64, dtype=torch.float, device=device)
    for k in range(Ncos) :
        cosvec[k,mer] =  tcoses[k, torch.arange(len(mer), dtype=torch.long, device=device)]
    dists[1:,:] = torch.sum(torch.unsqueeze(cosvec,dim=1)*tcosvecs,dim=2) # all cosine distances
    for k in range(Ncos+1) :
        allrank[k,:] = prank(dists[k,:], descending=True) # rank them all
    offset = 1.
    fm = torch.prod(offset+allrank,dim=0)
    fmargsort = torch.argsort(fm)
    # Levenshtein distance
    tseq1 = torch.tensor(seq,device=device)
    tseq2 = tallseqs[fmargsort[:Ntriage],:]
    ans = mydist(tseq1,tseq2)
    ia = torch.argmin(ans) # index into fmargsort of best
    ibest = fmargsort[ia] # index of best codeword in codes
    best[j] = (ibest if ans[ia] <= Nthresh else -1) # erasures returned as -1

print("done with decoding.")

Typical output above is\
`done with decoding.
CPU times: user 50.9 s, sys: 11.5 ms, total: 51 s
Wall time: 50.9 s`

If we were decoding real data (with the right answers not known) we would be done. The tensor `best` contains the index of the codeword of each read, or -1 if it is an erasure (i.e., not accurately readable). But in simulation we know the right answers and can check them.

**Next: Check simulation's recall and precision:**

In [ ]:
# DECODING Step 3a (for simulations done within this notebook).
# If there is an answers file, compute precision and recall of the decode.

answersfilename = "/tmp/randomcode_1e6_34_sim_answers" # user set to output file path
answers = genfromtxt(answersfilename, dtype=int)
best = best.cpu().numpy()
ngood = sum(logical_and(answers[:Qq] == best[:Qq], best[:Qq] >= 0))
nbad = sum(logical_and(answers[:Qq] != best[:Qq], best[:Qq] >= 0))
nerase = sum(best[:Qq] < 0)
recall = 1. - nerase / Qq
precision = ngood/(ngood + nbad + 1.e-30)

print(f"In {Qq} decodes, {ngood} were correct, {nbad} were wrong, {nerase} were erasures.")
print(f"precision = {precision:.4f}, recall = {recall:.4f}.")


For the "very large" error rates (0.05,0.05,0.10) typical output is\
`In 10000 decodes, 6634 were correct, 5 were wrong, 3361 were erasures.
precision = 0.9992, recall = 0.6639`\
For the "large" error rates (0.0333,0.0333,0.0333) typical output is\
`In 10000 decodes, 9751 were correct, 1 were wrong, 248 were erasures.
precision = 0.9999, recall = 0.9752.`

**Next: Checking simulated reads processed by the Python batch file (see Github explanation):**

In [ ]:
# DECODING Step 3b (for simulations processed by the standalone Python file barcode_batch.py).
from numpy import *
import re

answersfilename = "/tmp/randomcode_1e6_34_sim_1e6answers" # user to set
decodefilestem = "/tmp/batch_sim_output_" # user to set
decodefilesuffixes = ["1of4","2of4","3of4","4of4"] # user to set for files spanning answers

answers = genfromtxt(answersfilename, dtype=int)
totlines = len(answers)
print(f"found {totlines} answers")

for suffix in decodefilesuffixes :
    #convert suffixes to line numbers in answer file
    num,denom = re.match('([0-9]+)of([0-9]+)',suffix).groups()
    startline = int(totlines*(float(num)-1.)/float(denom)) # exactly same as barcode_batch.py
    endline = int(totlines*float(num)/float(denom)) # ditto
    print(f"comparing segment {suffix} to answer lines {startline}:{endline}:")
    # read the decodes and compare
    decodefile = decodefilestem + suffix
    decodes = genfromtxt(decodefile, dtype=int)
    ngood = sum(logical_and(answers[startline:endline] == decodes, decodes >= 0))
    nbad = sum(logical_and(answers[startline:endline] != decodes, decodes >= 0))
    nerase = sum(decodes < 0)
    recall = 1. - nerase / len(decodes)
    precision = ngood/(ngood + nbad + 1.e-30)
    print(f"     In {len(decodes)} decodes, {ngood} were correct, {nbad} were wrong, {nerase} were erasures.")
    print(f"     precision = {precision:.4f}, recall = {recall:.4f}.")
